In [ ]:
from openai import OpenAI
import requests
import json

In [ ]:
with open('api_keys.json') as f:
    keys = json.load(f)
    yelp_api_key = keys['yelp_api_key']
    openai_api_key = keys['openai_api_key']

client = OpenAI(api_key=openai_api_key)

In [ ]:
def generate_review(restaurant_name, reviews):
    """function to generate a new review with OpenAI's GPT-3 model based on a
    list of reviews"""
    prompt = f'Generate a new review for {restaurant_name} based on the '\
        + 'following reviews:\n'
    for review in reviews:
        prompt += f'- {review}\n'
    prompt += 'Make the new review about a specific dish that was ' \
        + 'mentionned in the given reviews.\n'
    prompt += 'New review:'
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=1024,
    )
    review_text = response.choices[0].message.content.strip()
    return review_text

In [ ]:
location = 'Westwood, Los Angeles'
# Define the URL for the Yelp Fusion API to search for restaurants
url = f'https://api.yelp.com/v3/businesses/search?term=restaurants&location={location}'
# Send a request to the API and get the response
headers = {'Authorization': f'Bearer {yelp_api_key}'}
response = requests.get(url, headers=headers)

# Extract the list of restaurants from the response
list_of_restaurants = response.json()
restaurants = list_of_restaurants['businesses']
json.dump(list_of_restaurants, open("list_of_restaurants.json", 'w'), indent=2)

In [ ]:
# Create a dictionary to store the new reviews for each restaurant
new_reviews = {}
true_reviews = {}

# Loop through each restaurant and get its reviews
for restaurant in restaurants:
    # Define the URL for the Yelp Fusion API to get reviews for the restaurant
    business_id = restaurant['id']
    url = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'

    # Send a request to the API and get the response
    response = requests.get(url, headers=headers)
    data = response.json()

    # Extract the list of reviews and their ratings from the response
    reviews = data['reviews']
    review_texts = []
    review_ratings = []

    for i, review in enumerate(reviews):
        text = review['text']
        rating = review['rating']

        review_texts.append(text)
        review_ratings.append(rating)

        # store the true reviews
        true_reviews[f"{restaurant['name']}-{i}"] = {
            'url': restaurant['url'],
            'review': text,
            'rating': rating
        }

    # Calculate the average rating for the restaurant
    avg_rating = int(sum(review_ratings) / len(review_ratings))

    # Generate a new review with OpenAI's GPT-3 model
    for i in range(4):

        new_review = generate_review(restaurant['name'], review_texts)
        # Add the new review and rating to the new_reviews dictionary
        new_reviews[f"{restaurant['name']}-{i}"] = {
            'url': restaurant['url'],
            'review': new_review,
            'rating': avg_rating
        }

In [ ]:
json.dump(new_reviews, open("new_reviews.json", 'w'), indent=2)
json.dump(true_reviews, open("true_reviews.json", 'w'), indent=2)